#### Google Palm LLM & API key setup

In [1]:
from langchain_google_genai import GoogleGenerativeAI
import os

from dotenv import load_dotenv
load_dotenv()

llm = GoogleGenerativeAI(model="gemini-pro",google_api_key=os.getenv("GOOGLE_API_KEY"))
llm

GoogleGenerativeAI(model='gemini-pro', google_api_key=SecretStr('**********'), client=genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
))

In [2]:
print(llm("Write few lines on Gen AI"))

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


**Generative Artificial Intelligence (Gen AI)**

Gen AI is a cutting-edge subset of AI that generates new, original content from scratch. Unlike traditional AI systems that rely on pre-trained data, Gen AI models can create novel images, text, music, and other forms of media.

Gen AI models leverage advanced techniques such as deep learning, neural networks, and generative adversarial networks (GANs) to generate realistic and diverse content that mimics human creativity. These models can produce highly detailed images, write coherent text, and compose music with remarkable accuracy.

Gen AI holds immense potential for transformative applications in various industries. It can revolutionize content creation, entertainment, education, and even scientific research by automating tasks and generating unique and engaging content.


#### Connect with database and ask some basic questions

In [3]:
from langchain.utilities import SQLDatabase


In [4]:
db_user = "root"
db_password = "Joel1304"
db_host = "localhost"
db_name = "atliq_tshirts"

db = SQLDatabase.from_uri(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
1	1	10.00
2	2	15.00
3	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock

In [5]:

from langchain_experimental.sql import SQLDatabaseChain

In [6]:
from langchain.chains import create_sql_query_chain

In [7]:
chain = create_sql_query_chain(llm,db)
response = chain.invoke({"question":"How much is the price of the inventory for all small size t-shirts?"})
response

"```sql\nSELECT SUM(`price` * `stock_quantity`) AS `total_price`\nFROM `t_shirts`\nWHERE `size` = 'S';\n```"

Above is the correct answer 👍🏼

In [8]:
cleaned_query = response.replace("```sql\n", "").replace("\n```", "").strip()
print(cleaned_query)

SELECT SUM(`price` * `stock_quantity`) AS `total_price`
FROM `t_shirts`
WHERE `size` = 'S';


In [9]:
result = db.run(cleaned_query)
print(result)

[(Decimal('14920'),)]


In [10]:
q2 = chain.invoke({"question":"I want to know How many t-shirts do we have left for nike in extra small size and white color?"})
cleaned_query2 = q2.replace("```sql\n", "").replace("\n```", "").strip()
print(cleaned_query2)

SELECT 
    `stock_quantity`
FROM 
    `t_shirts`
WHERE 
    `brand` = 'Nike'
    AND `size` = 'XS'
    AND `color` = 'White'
LIMIT 
    5;


In [11]:
result2 = db.run(cleaned_query2)
print(result2)

[(90,)]


In [12]:
def execute_query(question):
    try:
        response = chain.invoke({"question":question})
        print(response)
        print("#########################################################################")
        cleaned_query = response.replace("```sql\n", "").replace("\n```", "").strip()        
        print(cleaned_query)
        print("#########################################################################")
        result = db.run(cleaned_query)
        print(result)
    except:
        print(f"An error occured")

In [13]:
q1 = "How many t-shirts do we have left for nike in extra small size and white color?"
execute_query(q1)

```sql
SELECT SUM(`stock_quantity`) 
FROM `t_shirts` 
WHERE `brand` = 'Nike'
AND `color` = 'White'
AND `size` = 'XS';
```
#########################################################################
SELECT SUM(`stock_quantity`) 
FROM `t_shirts` 
WHERE `brand` = 'Nike'
AND `color` = 'White'
AND `size` = 'XS';
#########################################################################
[(Decimal('90'),)]


In [14]:
q2 = "How much is the total price of all small size t-shirts?"
execute_query(q2)

```sql
SELECT SUM(`t_shirts`.`price`) 
FROM `t_shirts` 
WHERE `t_shirts`.`size` = 'S';
```
#########################################################################
SELECT SUM(`t_shirts`.`price`) 
FROM `t_shirts` 
WHERE `t_shirts`.`size` = 'S';
#########################################################################
[(Decimal('352'),)]


In [15]:
q3 = "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"
execute_query(q3)

```sql
SELECT SUM(`t_shirts`.`price` * (1 - `discounts`.`pct_discount` / 100)) AS `revenue`
FROM `t_shirts`
JOIN `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id`
WHERE `t_shirts`.`brand` = 'Levi'
GROUP BY `t_shirts`.`brand`;
```
#########################################################################
SELECT SUM(`t_shirts`.`price` * (1 - `discounts`.`pct_discount` / 100)) AS `revenue`
FROM `t_shirts`
JOIN `discounts` ON `t_shirts`.`t_shirt_id` = `discounts`.`t_shirt_id`
WHERE `t_shirts`.`brand` = 'Levi'
GROUP BY `t_shirts`.`brand`;
#########################################################################
[(Decimal('38.750000'),)]


##### The above code is not giving us the desired output as per our requirements

- Lets try with the desired SQL code and check the output

In [16]:
q3_ = """select round(sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100))) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """
execute_query(q3_)

```sql
select round(sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100))) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
```
#########################################################################
select round(sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100))) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
#########################################################################
[(Decimal('22863'),)]


In [17]:
q4 = "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'"
execute_query(q4)

```sql
SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi';
```
#########################################################################
SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi';
#########################################################################
[(Decimal('23136'),)]


In [18]:
q5 = execute_query("How many white color Levi's t shirts we have available?")
q5

```sql
SELECT COUNT(*) AS `white_levi_count`
FROM t_shirts
WHERE `brand` = 'Levi' AND `color` = 'White';
```
#########################################################################
SELECT COUNT(*) AS `white_levi_count`
FROM t_shirts
WHERE `brand` = 'Levi' AND `color` = 'White';
#########################################################################
[(3,)]


#### The above code is not giving the required output

In [19]:
# lets try with the sql query

q5 = execute_query("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")
q5

```sql
SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White';
```
#########################################################################
SELECT SUM(`stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Levi' AND `color` = 'White';
#########################################################################
[(Decimal('148'),)]


# Few short learning

##### we will train our model with question, related SQL query and then the answer, in this way it will understand what we expect from the model to give us as the answer

In [28]:
few_shots_learning = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "90"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': "352"},
    {'Question': "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': "22863"} ,
     {'Question' : "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : "23136"},
    {'Question': "How many white color Levi's shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "148"
     }
]

### Creating Semantic Similarity Based example selector

- create embedding (numerical vectors) on the few_shots_learning
- Store the vectors in Chroma DB -
- Retrieve the the top most Semantically close answer from the vector store

In [21]:
# pip uninstall -y keras tensorflow transformers

In [22]:
!pip show transformers

Name: transformers
Version: 4.48.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: langchain-huggingface, sentence-transformers


In [25]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [29]:
to_vectorize = [" ".join(example.values()) for example in few_shots_learning]
to_vectorize

["How many t-shirts do we have left for Nike in XS size and white color? SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS' Result of the SQL query 90",
 "How much is the total price of the inventory for all S-size t-shirts? SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S' Result of the SQL query 352",
 "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)? SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n  Result of the SQL query 22863",
 "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount? SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi' Resu

In [31]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots_learning)

In [32]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

example_selector.select_examples({"Question": "How many Adidas T shirts I have left in my store?"})

[{'Answer': '90',
  'Question': 'How many t-shirts do we have left for Nike in XS size and white color?',
  'SQLQuery': "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': '23136',
  'Question': 'If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?',
  'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
  'SQLResult': 'Result of the SQL query'}]

In [33]:
### my sql based instruction prompt
mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.
"""

In [34]:
from langchain.prompts import FewShotPromptTemplate
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


### Setting up PromptTemplete using input variables

In [35]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)

In [36]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [37]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [38]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [39]:
new_chain("How many white color Levi's shirt I have?")

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How many white color Levi's shirt I have?
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('148'),)]
Answer:148
> Finished chain.


{'query': "How many white color Levi's shirt I have?", 'result': '148'}

Now this is working ok. Previously for this same question it was giving wrong answer because it did not use SUM clause around stock_quantity column

In [40]:
new_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:SELECT sum(price * stock_quantity) AS total_inventory_price
FROM t_shirts
WHERE size = 'S'
SQLResult: [(Decimal('14920'),)]
Answer:14920
> Finished chain.


{'query': 'How much is the price of the inventory for all small size t-shirts?',
 'result': '14920'}

In [41]:
new_chain("How much is the price of all white color levi t shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of all white color levi t shirts?
SQLQuery:SELECT sum(stock_quantity * price) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('5040'),)]
Answer:5040
> Finished chain.


{'query': 'How much is the price of all white color levi t shirts?',
 'result': '5040'}

In [42]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Nike'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('19553.300000'),)]
Answer:19553.3
> Finished chain.


{'query': 'If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '19553.3'}

In [43]:
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?
SQLQuery:SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Van Huesen'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('26183.600000'),)]
Answer:26183.6
> Finished chain.


{'query': 'If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?',
 'result': '26183.6'}

In [44]:
new_chain.run('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')



> Entering new SQLDatabaseChain chain...
How much revenue  our store will generate by selling all Van Heuson TShirts without discount?
SQLQuery:

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


SELECT SUM(`price` * `stock_quantity`) FROM `t_shirts` WHERE `brand` = 'Van Huesen'
SQLResult: [(Decimal('26816'),)]
Answer:26816
> Finished chain.


'26816'

In [45]:
!pip show langchain

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: langchain
Version: 0.2.7
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: aiohttp, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community, langchain-experimental


In [46]:
!pip show langchain-community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: langchain-community
Version: 0.2.7
Summary: Community contributed LangChain integrations.
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: aiohttp, dataclasses-json, langchain, langchain-core, langsmith, numpy, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [48]:
!pip show protobuf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: protobuf
Version: 4.25.3
Summary: 
Home-page: https://developers.google.com/protocol-buffers/
Author: protobuf@googlegroups.com
Author-email: protobuf@googlegroups.com
License: 3-Clause BSD License
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: 
Required-by: google-ai-generativelanguage, google-api-core, google-generativeai, googleapis-common-protos, grpcio-status, onnxruntime, opentelemetry-proto, proto-plus, streamlit, tensorboard, tensorflow
